# Caparezza Sentiment Analysis

In this notebook, a text mining process on the works of the Italian singer
**Caparezza** will be performed, with the intention of finding recurring themes
throughout his albums and songs.

Much of the inspiration for this project comes from [this repository](https://tm4ss.github.io/docs/index.html).

## Get the lyrics

In [ ]:
library(geniusr)
library(tidyverse)
library(tidytext)
library(quanteda)
library(quanteda.textstats)
library(udpipe)
library(wordcloud)
library(textdata)
library(reshape2)
library(igraph)

Lyrics for the songs can be obtained using the *geniusr* package, available
at https://github.com/ewenme/geniusr. Follow the instructions at that link
to set up an API key and use it to query the Genius lyrics database.

In [ ]:
Sys.setenv(GENIUS_API_TOKEN = "mjJVgXu4_yQNgYXFYV72Q9uXCIdTFFq3s3c8PPcUT6G1tlAedbpg0dpdZv6KJQW7") 

artist_id <- search_artist("Caparezza")$artist_id
artist_songs <- get_artist_songs(artist_id)

songs_ids <- c()
for (song in artist_songs$content) {
  song_id <- song$id
  songs_ids <- songs_ids %>% append(song_id)
}

songs_titles <- c()
songs_albums <- c()
songs_lyrics <- c()
for (i in 1:length(songs_ids)) {
  cat("Getting", i, "of", length(songs_ids), "id:", songs_ids[i], "\n")
  song <- get_song(songs_ids[i])$content
  song_title <- song$title
  song_album <- song$album$name
  song_lyrics <- list(get_lyrics_id(songs_ids[i]))
  # some songs have no album associated with them, so we won't consider them
  # additionally, some songs have no lyrics, we are excluding them too
  if (!is.null(song_album) & nrow(song_lyrics[[1]]) != 0){
    songs_titles <- songs_titles %>% append(song_title)
    songs_albums <- songs_albums %>% append(song_album)
    songs_lyrics <- songs_lyrics %>% append(song_lyrics) 
  }
}

# collapse all lyrics lines into a single text
for (i in 1:length(songs_lyrics)){
  songs_lyrics[[i]] <- songs_lyrics[[i]]$line %>% paste(collapse = "\n")
}

songs_lyrics <- unlist(songs_lyrics)

songs <- data.frame(title = songs_titles, album = songs_albums, lyrics = songs_lyrics)

Now we have got a dataframe with data about each individual song.
Let's look at the first element.

In [ ]:
songs %>% head(1)

Songs are ordered alphabetically by the title, and the first one appears
to be *Abiura Di Me*.

Let's do some cleaning: we are going to ignore alternative versions of the same
song, such as live versions, radio edits and remixes.
We are also keeping only the main albums, ignoring specials or compilations.

In [ ]:
albums <- c("?! (Caparezza ?!)", "Verità Supposte", "Habemus Capa",
            "Le Dimensioni Del Mio Caos", "Il Sogno Eretico", "Museica",
            "Prisoner 709", "Exuvia")
songs <- songs %>%
  filter(!grepl("Live|Radio Edit|Radio Version|Remix|RMX", title) & album %in% albums)

Finally, we order songs chronologically by the album, then add an id for each song.

In [ ]:
songs <- songs %>% arrange(match(album, albums), title)
doc_ids <- vector()
for(i in 1:nrow(songs)){
  id <- paste("doc", toString(i), sep = "")
  doc_ids <- doc_ids %>% append(id)
}
songs <- songs %>% mutate(doc_id = doc_ids, .before = 1)

Let's check the number of songs for each album.

In [ ]:
table(songs$album) %>% as.data.frame() %>% arrange(-Freq)

The frequencies are correct, as every Caparezza fan will recognize.

## Text pre-processing

Now we start with some text pre-processing: let's create a corpus from the
songs' lyrics.

In [ ]:
caparezza_corpus <- corpus(songs$lyrics, docnames = songs$id)
summary(caparezza_corpus)

There is a total of 129 documents and for each one the total number of tokens
is displayed. A *token* is a single occurrence of a word in the document.

In [ ]:
cat(as.character(caparezza_corpus[1]))

The first document of the corpus is the song "Cammina Solo" from *?!*, since
we have ordered songs by album and then by title.

One of the main steps of lexical analysis is the removal of punctuation marks,
numbers and symbols which are not useful towards the text interpretation.

The *quanteda* package offers some useful tools for these operations.

In [ ]:
corpus_tokens <- caparezza_corpus %>%
  quanteda::tokens(remove_punct = TRUE, remove_numbers = TRUE, remove_symbols = TRUE) %>%
  tokens_tolower()

Additionally, we would like to lemmatise: that is, to consider the *lemma* from
which a word originates, instead of the world itself. For example, the
infinitive forms of the verbs or the standard singular masculine adjective form
are sufficient to express a concept, so we can safely use those instead of their
derivatives.

In [ ]:
txt <- sapply(corpus_tokens, FUN=function(x) paste(x, collapse = "\n"))
udpipe_download_model(language = "italian-isdt", model_dir = "resources/")
lang_model <- udpipe_load_model(file = "resources/italian-isdt-ud-2.5-191206.udpipe")
outL <- udpipe_annotate(lang_model, x = txt, tokenizer = "vertical", trace = TRUE) %>%
  as.data.frame()
it_stopwords <- readLines("https://raw.githubusercontent.com/stopwords-iso/stopwords-it/master/stopwords-it.txt")
outL <- outL %>% filter(!(token %in% it_stopwords) & !(lemma %in% it_stopwords))

We have made use of the *UDPipe* library annotation function.

In the process, we have also removed *stopwords*, words that do not bring
any meaningful addition to our texts, but are only necessary to connect other
words and respect syntactic rules.

Let's have a look at random sample of five elements of the output.

In [ ]:
outL %>% select(doc_id, token, lemma, upos) %>% sample_n(5)

To further enhance our analysis, let's focus only on nouns, proper nouns,
adjectives and verbs.

In [ ]:
outL_reduced <- outL %>% filter(upos %in% c("NOUN", "PROPN", "ADJ", "VERB"))

Now we create a new corpus with the lemmatized lyrics.

In [ ]:
# fct_inorder preserves original order of the column
lemmatized_lyrics <- outL_reduced %>% group_by(doc_id = fct_inorder(doc_id)) %>%
  summarise(lemmatized = paste(lemma, collapse = " "))
songs <- songs %>% right_join(lemmatized_lyrics, by = "doc_id")

caparezza_corpus <- songs$lemmatized %>% corpus(docnames = songs$id)

One final step for text-processing consists in handling collocations, or
multi-word units(MWUs). A collocation is a set of two or more
words which are closely related and are often used together to express a single
concept.

They can be identified through statistical, objective methods, like in this case.

In [ ]:
collocations <- caparezza_corpus %>% tokens() %>% textstat_collocations %>%
  arrange(-count) %>% head(10)

Looking at the collocations found by the function, we decide to take action on
two of them, which seem to be clearly meant to be used together.

In [ ]:
DTM <- caparezza_corpus %>% tokens() %>% tokens_compound(collocations[c(4,6),]) %>%
  tokens_remove("") %>% dfm()

## Lexical Analysis

In the last step, we have created an object called DTM. This is the document-term
matrix of our corpus: it's a matrix which contains documents on its rows and
terms on its columns. Every element of the matrix describes the number of
occurrencies of the corresponding term inside the corresponding document.

In [ ]:
DTM
DTM %>% dim()

We have got a total of 129 documents and 8679 unique terms in our corpus.

Let's build a frequencies table for our terms and look at the most recurring ones.

In [ ]:
words <- colnames(DTM)
freqs <- colSums(DTM)
wordlist <- data.frame(words, freqs)
wordlist %>% arrange(-freqs) %>% head()

And now let's look at some useful quantities.

In [ ]:
corpus_size <- sum(wordlist$freqs)
corpus_size

vocabulary_size <- nrow(wordlist)
vocabulary_size

words_occurrencies <- wordlist %>% group_by(freqs) %>% summarise(vK = n()) %>% arrange(-vK)
words_occurrencies

lexicon_width <- vocabulary_size/corpus_size
lexicon_width
language_refinement <- words_occurrencies$vK[1] / colSums(words_occurrencies)[2]
language_refinement

Words that appear only once in the entire corpus are known as *hapaxes*.
*Lexicon width* is the percentage of unique words in the total number of words
used in the corpus.
*Language refinement* is the percentage of hapaxes in the total number of words
used in the corpus.

## Data visualization

In order to visualize the most frequent terms, we can use a *word cloud*, which
portrays the terms in the corpus with different sizes depending on their relative
frequency.

In [ ]:
par(mar=c(1,1,0.5,1))
wordcloud(words = wordlist$words, freq = wordlist$freqs, scale = c(3.5, 0.35),
          max.words = 50, random.order = F,
          colors = RColorBrewer::brewer.pal(name = "Dark2", n = 4))
text(0.5, 1, "wordcloud with TF ponderation", font = 2)

It could be interesting to know in which songs a given word appears, especially
if it seems an odd one.
We can use this function.

In [ ]:
count <- 1
for(i in songs$lyrics){
  if(grepl("mamma", i)){
    print(songs$title[[count]])
  }
  count <- count + 1
}

## TF-IDF ponderation

It is often necessary to weight terms in the corpus on the basis of their relative
importance. For example, a term which occurs frequently in a document has a strong
relevance for that document, but a term which occurs frequently in many different
documents is less informative for any single document and should be treated as
less relevant.

To account for this situation, we are going to use the term frequency-inverse
document frequency (TF-IDF) weighting.

In [ ]:
tf_idf <- dfm_tfidf(DTM)
freqs_tf_idf <- colSums(tf_idf)
words_tf_idf <- colnames(tf_idf)
wordlist_tf_idf <- data.frame(words = words_tf_idf, freqs = freqs_tf_idf)
wordlist_tf_idf %>% arrange(-freqs) %>% head(10)

These are the most common terms after the applied weighting.

Let's have a look at the new word cloud.

In [ ]:
par(mar=c(1,1,0.5,1))
wordcloud(words = wordlist_tf_idf$words, freq = wordlist_tf_idf$freqs,
          scale = c(3.5, 0.35), max.words = 50, random.order = F,
          colors = RColorBrewer::brewer.pal(name = "Dark2", n = 4))
text(0.5, 1, "wordcloud with TF-IDF ponderation", font = 2)

## Group by albums

Let's redo some of our previous analysis, this time considering the entire albums
as documents, rather than individual songs.

In [ ]:
lemmatized_lyrics_by_album <- songs %>% group_by(album) %>%
  summarise(lemmatized = paste(lemmatized, collapse = " ")) %>% arrange(match(album, albums))
corpus_album <- lemmatized_lyrics_by_album$lemmatized %>% corpus()

DTM_album <- corpus_album %>% tokens() %>% dfm()
DTM_album
docnames(DTM_album) <- lemmatized_lyrics_by_album$album
wordlist_album <- DTM_album %>% as.matrix() %>% t()
par(mar=c(0,0,0,0))
comparison.cloud(wordlist_album, scale = c(2, 1), max.words = 50, random.order = F,
                 title.size = 1, colors = RColorBrewer::brewer.pal(name = "Dark2", n = 8))
text(0.5, 1, "comparison cloud by album", font = 2)

We can recognize some iconic songs from each album by looking at the words:
*conflitto* from *Il Conflitto* (*?!*),
*secessionista* from *Inno Verdano* (*Habemus Capa*),
*campione* from *Campione dei Novanta* (*Exuvia*), and more.

## Co-occurrence analysis

Next, we are going to analyze the co-occurrence of words in order to find
terms which tend to appear in the same documents.

In [ ]:
binDTM <- DTM %>% dfm_trim(min_docfreq = 10) %>% dfm_weight("boolean")
coocCounts <- t(binDTM) %*% binDTM
as.matrix(coocCounts[100:102, 100:102])

For example, the words *libro* and *leggere* appear together in four documents.
The diagonal elements of the matrix are the total occurrencies of the word.

Let's calculate some co-occurrence measurements for the words *lavoro* and then
*leggere*: Mutual Information, Dice, and Log-Likelihood.

In [ ]:
coocTerm <- "lavoro"
k <- nrow(binDTM)
ki <- sum(binDTM[, coocTerm])
kj <- colSums(binDTM)
names(kj) <- colnames(binDTM)
kij <- coocCounts[coocTerm, ]

mutualInformationSig <- log(k * kij / (ki * kj))
mutualInformationSig <- mutualInformationSig[order(mutualInformationSig, decreasing = TRUE)]

dicesig <- 2 * kij / (ki + kj)
dicesig <- dicesig[order(dicesig, decreasing=TRUE)]

logsig <- 2 * ((k * log(k)) - (ki * log(ki)) - (kj * log(kj)) + (kij * log(kij)) 
               + (k - ki - kj + kij) * log(k - ki - kj + kij) 
               + (ki - kij) * log(ki - kij) + (kj - kij) * log(kj - kij) 
               - (k - ki) * log(k - ki) - (k - kj) * log(k - kj))
logsig <- logsig[order(logsig, decreasing=T)]

resultOverView <- data.frame(
  names(sort(kij, decreasing=T)[1:10]), sort(kij, decreasing=T)[1:10],
  names(mutualInformationSig[1:10]), mutualInformationSig[1:10], 
  names(dicesig[1:10]), dicesig[1:10], 
  names(logsig[1:10]), logsig[1:10],
  row.names = NULL)
colnames(resultOverView) <- c("Freq-terms", "Freq", "MI-terms", "MI", "Dice-Terms", "Dice", "LL-Terms", "LL")
print(resultOverView)

In [ ]:
coocTerm <- "leggere"
k <- nrow(binDTM)
ki <- sum(binDTM[, coocTerm])
kj <- colSums(binDTM)
names(kj) <- colnames(binDTM)
kij <- coocCounts[coocTerm, ]

mutualInformationSig <- log(k * kij / (ki * kj))
mutualInformationSig <- mutualInformationSig[order(mutualInformationSig, decreasing = TRUE)]

dicesig <- 2 * kij / (ki + kj)
dicesig <- dicesig[order(dicesig, decreasing=TRUE)]

logsig <- 2 * ((k * log(k)) - (ki * log(ki)) - (kj * log(kj)) + (kij * log(kij)) 
               + (k - ki - kj + kij) * log(k - ki - kj + kij) 
               + (ki - kij) * log(ki - kij) + (kj - kij) * log(kj - kij) 
               - (k - ki) * log(k - ki) - (k - kj) * log(k - kj))
logsig <- logsig[order(logsig, decreasing=T)]

resultOverView <- data.frame(
  names(sort(kij, decreasing=T)[1:10]), sort(kij, decreasing=T)[1:10],
  names(mutualInformationSig[1:10]), mutualInformationSig[1:10], 
  names(dicesig[1:10]), dicesig[1:10], 
  names(logsig[1:10]), logsig[1:10],
  row.names = NULL)
colnames(resultOverView) <- c("Freq-terms", "Freq", "MI-terms", "MI", "Dice-Terms", "Dice", "LL-Terms", "LL")
print(resultOverView)

## Co-occurrence visualization

To visualize co-occurrence for a single term, we may use a graph which displays
the terms co-occurrence network (including secondary co-occurrence levels).

In [ ]:
source("resources/calculateCoocStatistics.R")
numberOfCoocs <- 10
coocTerm <- "libro"
coocs <- calculateCoocStatistics(coocTerm, binDTM, measure="LOGLIK")
print(coocs[1:numberOfCoocs])
resultGraph <- data.frame(from = character(), to = character(), sig = numeric(0))
tmpGraph <- data.frame(from = character(), to = character(), sig = numeric(0))

# Fill the data.frame to produce the correct number of lines
tmpGraph[1:numberOfCoocs, 3] <- coocs[1:numberOfCoocs]
# Entry of the search word into the first column in all lines
tmpGraph[, 1] <- coocTerm
# Entry of the co-occurrences into the second column of the respective line
tmpGraph[, 2] <- names(coocs)[1:numberOfCoocs]
# Set the significances
tmpGraph[, 3] <- coocs[1:numberOfCoocs]

# Attach the triples to resultGraph
resultGraph <- rbind(resultGraph, tmpGraph)

# Iteration over the most significant numberOfCoocs co-occurrences of the search term
for (i in 1:numberOfCoocs){
  
  # Calling up the co-occurrence calculation for term i from the search words co-occurrences
  newCoocTerm <- names(coocs)[i]
  coocs2 <- calculateCoocStatistics(newCoocTerm, binDTM, measure="LOGLIK")
  
  #print the co-occurrences
  coocs2[1:10]
  
  # Structure of the temporary graph object
  tmpGraph <- data.frame(from = character(), to = character(), sig = numeric(0))
  tmpGraph[1:numberOfCoocs, 3] <- coocs2[1:numberOfCoocs]
  tmpGraph[, 1] <- newCoocTerm
  tmpGraph[, 2] <- names(coocs2)[1:numberOfCoocs]
  tmpGraph[, 3] <- coocs2[1:numberOfCoocs]
  
  #Append the result to the result graph
  resultGraph <- rbind(resultGraph, tmpGraph[2:length(tmpGraph[, 1]), ])
}

# Sample of some examples from resultGraph
resultGraph[sample(nrow(resultGraph), 6), ]


# set seed for graph plot
set.seed(1)

# Create the graph object as undirected graph
graphNetwork <- graph.data.frame(resultGraph, directed = F)

# Identification of all nodes with less than 2 edges
verticesToRemove <- V(graphNetwork)[degree(graphNetwork) < 2]
# These edges are removed from the graph
graphNetwork <- delete.vertices(graphNetwork, verticesToRemove) 

# Assign colors to nodes (search term blue, others orange)
V(graphNetwork)$color <- ifelse(V(graphNetwork)$name == coocTerm, 'cornflowerblue', 'orange') 

# Set edge colors
E(graphNetwork)$color <- adjustcolor("DarkGray", alpha.f = .5)
# scale significance between 1 and 10 for edge width
E(graphNetwork)$width <- scales::rescale(E(graphNetwork)$sig, to = c(1, 10))

# Set edges with radius
E(graphNetwork)$curved <- 0.15 
# Size the nodes by their degree of networking (scaled between 5 and 15)
V(graphNetwork)$size <- scales::rescale(log(degree(graphNetwork)), to = c(5, 15))

# Define the frame and spacing for the plot
par(mai=c(0,0,1,0)) 

# Final Plot
plot(
  graphNetwork,             
  layout = layout.fruchterman.reingold, # Force Directed Layout 
  main = paste(coocTerm, ' Graph'),
  vertex.label.family = "sans",
  vertex.label.cex = 0.8,
  vertex.shape = "circle",
  vertex.label.dist = 0.5,          # Labels of the nodes moved slightly
  vertex.frame.color = adjustcolor("darkgray", alpha.f = .5),
  vertex.label.color = 'black',     # Color of node names
  vertex.label.font = 2,            # Font of node names
  vertex.label = V(graphNetwork)$name,      # node names
  vertex.label.cex = 1 # font size of node names
)

## Sentiment analysis

We are now going to perform a sentiment analysis on the lyrics of the songs.
At [this link][https://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm), you
can download NRC emotion lexicons in different languages. It allows you to
catalogue words in positive / negative classes and even emotions, as we will see
later.

In [ ]:
sentiment_lexicon <- read.table("resources/Italian-NRC-EmoLex.txt",
                                header = TRUE, sep = "\t")
sentiment_lexicon_corpus <- sentiment_lexicon %>% filter(Italian.Word %in% colnames(DTM))
positive_terms <- sentiment_lexicon_corpus %>% filter(positive == 1) %>%
  select(Italian.Word) %>% pull()
negative_terms <- sentiment_lexicon_corpus %>% filter(positive == 0) %>%
  select(Italian.Word) %>% pull()
counts_positive <- rowSums(DTM[, positive_terms])
counts_negative <- rowSums(DTM[, negative_terms])
counts_all_terms <- rowSums(DTM)
relative_sentiment_frequencies <- data.frame(
  positive = counts_positive / counts_all_terms,
  negative = counts_negative / counts_all_terms
)
sentiments_by_album <- aggregate(relative_sentiment_frequencies,
                                 by = list(album = songs$album), mean)

head(sentiments_by_album)
df_sentiment <- melt(sentiments_by_album, id.vars = "album")
ggplot(data = df_sentiment, aes(x = album, y = value, fill = variable)) + 
  geom_bar(stat="identity", position="stack") + coord_flip()

Here we have catalogued every single word, then grouped words by albums and
visualized the sentiment distribution for each album. We see that the sentiment
is mostly negative for every single one. It has probably to do with the topics
that appear in Caparezza's songs, who is notoriously a socially engaged singer
and therefore often criticizes the hypocrisy of society.

It would be interesting to know if there is at least one song with more positive
words than negative ones. Let's find out.

In [ ]:
positive_songs <- aggregate(
  relative_sentiment_frequencies, by = list(album = songs$title),
  mean) %>% filter(positive > negative)
positive_songs

Here they are. *Chi se ne frega della musica*, *Fugadà* and *Uomini di molta fede*
have been classified as mostly positive songs.

## Emotion analysis

For the last part of our analysis, we are going to look at the emotions,
specifically anger, fear, joy and sadness, using the same NCR lexicon as before.

In [ ]:
anger_terms <- sentiment_lexicon_corpus %>% filter(anger == 1) %>%
  select(Italian.Word) %>% pull()
fear_terms <- sentiment_lexicon_corpus %>% filter(fear == 1) %>%
  select(Italian.Word) %>% pull()
joy_terms <- sentiment_lexicon_corpus %>% filter(joy == 1) %>%
  select(Italian.Word) %>% pull()
sadness_terms <- sentiment_lexicon_corpus %>% filter(sadness == 1) %>%
  select(Italian.Word) %>% pull()

counts_anger <- rowSums(DTM[, anger_terms])
counts_fear <- rowSums(DTM[, fear_terms])
counts_joy <- rowSums(DTM[, joy_terms])
counts_sadness <- rowSums(DTM[, sadness_terms])

relative_emotion_frequencies <- data.frame(
  anger = counts_anger / counts_all_terms,
  fear = counts_fear / counts_all_terms,
  joy = counts_joy / counts_all_terms,
  sadness = counts_sadness / counts_all_terms
)

emotions_by_album <- aggregate(relative_emotion_frequencies,
                                 by = list(album = songs$album), mean)

head(emotions_by_album)

df_emotions <- melt(emotions_by_album, id.vars = "album")
ggplot(data = df_emotions, aes(x = album, y = value, fill = variable)) + 
  geom_bar(stat="identity", position="stack") + coord_flip()

Anger and fear are the prevailing emotions and they are mostly noticeable in
the *Museica* album. The album where joy shares a larger percentage is *Exuvia*.